# Index MS Marco dateset

In [4]:
from elasticsearch import Elasticsearch
from typing import Dict, List, Optional
import json
from trec_tools.trec_car.read_data import *
import ipytest
import pytest

ipytest.autoconfig()

In [10]:
INDEX_SETTINGS = {
    "mappings": {  
    "properties": {
        "passage": {"type": "text", "term_vector": "yes", "analyzer": "english"},
        "origin": {"type": "text"}
    }
    }
}

In [6]:
INDEX_NAME = "cast_base"

In [5]:
es = Elasticsearch()

In [13]:
# if es.indices.exists(INDEX_NAME):
#     es.indices.delete(index=INDEX_NAME)
# es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

C:\Users\jafar\anaconda3\envs\DAT640\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.
C:\Users\jafar\anaconda3\envs\DAT640\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\jafar\anaconda3\envs\DAT640\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters direc

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cast_base'}

In [8]:
def index_msmarco(filepath: str, es: Elasticsearch, index: str, car:bool = False) -> None:
    """Indexes documents from JSONL file."""
    bulk_data = []
    with open(filepath, "r",encoding="utf-8") as docs:
        for i,line in enumerate(docs):
            s = line.split("\t")
            id = s[0]
            passage = s[1].strip()
            bulk_data.append(
                {"index":{"_index": index, "_id": id}}
            )
            bulk_data.append({"passage":passage,"origin":"msmarco"})
            if i%2000==0:
                es.bulk(index=index, body=bulk_data, refresh=True)
                bulk_data = []
        es.bulk(index=index,body=bulk_data, refresh=True)
        bulk_data = []
        para_cnt = 0
    if car:
        for para in iter_paragraphs(open('../data/dedup.articles-paragraphs.cbor', 'rb')):
            para_cnt += 1
            bulk_data.append({"index":{"_index": index, "_id": para.para_id}})
            bulk_data.append({"passage":para.get_text(),"origin":"car"})
            if para_cnt%2000==0:
                es.bulk(index=index, body=bulk_data, refresh=True)
                bulk_data = []
    return "finished"

In [15]:
index_msmarco(filepath = "../data/collection.tsv", es = es, index = INDEX_NAME, car=False)

'finished'

In [7]:
es.get(index=INDEX_NAME, id="8840824")

C:\Users\jafar\anaconda3\envs\DAT640\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'cast_base',
 '_type': '_doc',
 '_id': '8840824',
 '_version': 1,
 '_seq_no': 8840824,
 '_primary_term': 1,
 'found': True,
 '_source': {'passage': 'H 2 and C are the reducing agents(note: H 2 is oxidized, so CuO is an oxidizing agent) Oxidation-Reduction Reactions In all cases: â\x80¢ If something is oxidized, something must be reduced. â\x80¢ Redox reactions move e - .2Ag + (aq) + Cu(s) 2 Ag(s) + Cu 2+ (aq) â\x80¢ OOxxidation-reduction = redox.Here: â\x80¢ Cu changes to Cu 2+ .',
  'origin': 'msmarco'}}

In [3]:
with open("../data/collection.tsv", "r",encoding="utf-8") as docs:
    for i,line in enumerate(docs):
        s = line.split("\t")
        id = s[0]
        passage = s[1].strip()
print(id, passage)


8841822 View full size image. Behind the scenes of the dazzling light shows that spectators ooh and ahh at on the Fourth of July, are carefully crafted fireworks. Whether red, white and blue fountains or purple sparklers, each firework is packed with just the right mix of chemicals to create these colorful lights. Inside each handmade firework are small packets filled with special chemicals, mainly metal salts and metal oxides, which react to produce an array of colors.
